In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import datetime
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Bidirectional, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization, Input, concatenate, K, Reshape, LSTM, CuDNNLSTM
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.utils.np_utils import to_categorical
from keras.losses import *
from sklearn.preprocessing import LabelEncoder, minmax_scale, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import xml.etree.ElementTree as ET
import sklearn
import itertools
import cv2
import scipy
import os
import csv
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
%matplotlib inline
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Objective : Calculate sales/hour for a given system of system and manager scheduled hours
""""The solution should also identify the impact of any
changes made by the manager on the system
generated schedule and determine which
managers make better changes to the system
generated schedule or more generally, what
factors result in better schedule adjustments by
managers vs changes that result in worse / non-
optimal labor schedule & costs.
Objectives : Maximise sales hour
"""

In [ ]:
df = pd.read_csv('../input/BITS AIC 2019 - Reflexis Raw Dataset.csv')
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.set_index('DATE')
df.head()

In [ ]:
null_data = df[df.isnull().any(axis=1)]
#Clean dataframe
if(df.isnull().values.ravel().sum() > 0):
    df = df[pd.notnull(df['EPS'])]    


In [ ]:
df.head()

In [ ]:
print('Max sales = ', df['SALES_ACTUAL'].max())
print('Number of records =', len(df))
df.head()

In [ ]:

 #Max profit ever made is the max profit it will ever make, Min possible sales made is the minimum it was ever mode(Since we don't have sufficient parameters)


In [ ]:
df1 = df.groupby('STORE')
_temp = []
for store_no, store_data in df1:
    data = df1.get_group(store_no).copy()
    data['SALES_DELTA_NORM'] = minmax_scale(data['SALES_ACTUAL'], feature_range=(-1, 1))
    """scalar = MinMaxScalar(feature_range=(-1,1))
    data['SALES_DELTA_NORM'] = scalar.fit_transform(data['SALES_ACTUAL'])"""
    for i in data['SALES_DELTA_NORM'].values:
        _temp.append(i)
df['SALES_DELTA_NORM'] = _temp
df1 = df.groupby('STORE')

In [ ]:
class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)
        
class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)

def plotKerasLearningCurve():
    plt.figure(figsize=(10,5))
    metrics = np.load('logs.npy')[()]
    filt = ['mean_absolute_percentage_error'] # try to add 'loss' to see the loss learning curve
    for k in filter(lambda x : np.any([kk in x for kk in filt]), metrics.keys()):
        l = np.array(metrics[k])
        plt.plot(l, c= 'r' if 'val' not in k else 'b', label='val' if 'val' in k else 'train')
        x = np.argmin(l)
        y = l[x]
        plt.scatter(x,y, lw=0, alpha=0.25, s=100, c='r' if 'val' not in k else 'b')
        plt.text(x, y, '{} = {:.4f}'.format(x,y), size='15', color= 'r' if 'val' not in k else 'b')   
    plt.legend(loc=4)
    plt.axis([0, None, None, None]);
    plt.grid()
    plt.xlabel('Number of epochs')
    plt.ylabel('MAPE')
    
def plot_learning_curve(history):
    plt.figure(figsize=(8,8))
    plt.subplot(1,2,1)
    plt.plot(history.history['mean_absolute_percentage_error'])
    plt.plot(history.history['val_mean_absolute_percentage_error'])
    """plt.plot(scalar.inverse_transform(history.history['loss']))
    plt.plot(scalar.inverse_transform(history.history['val_loss']))"""
    plt.title('model MAPE')
    plt.ylabel('MAPE')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./accuracy_curve.png')
    #plt.clf()
    # summarize history for loss
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./loss_curve.png')

In [ ]:
#def f1(dfin, store_no, epochs):
def f1(data, store_no, epochs):
    #predict sales given date, store number, system scheduled hours and manager scheduled hours
   # data = dfin.get_group(store_no)
#    batch_size = 16
    batch_size = 256
    split_date = '2018-01-01'
    X_train, Y_train, X_test, Y_test = data[data.index < split_date][['STORE','MANAGER_SCHED_HOURS','SYSTEM_SCHED_HOURS']], data[data.index < split_date]['SALES_DELTA_NORM'], data[data.index> split_date][['STORE','MANAGER_SCHED_HOURS','SYSTEM_SCHED_HOURS']], data[data.index > split_date]['SALES_DELTA_NORM']
    #X_train, X_test, Y_train, Y_test = train_test_split(data[['STORE','MANAGER_SCHED_HOURS','SYSTEM_SCHED_HOURS']],data['SALES_ACTUAL'], shuffle=False)
    X_train = X_train.values.reshape(X_train.shape[0],1,X_train.shape[1])
    X_test = X_test.values.reshape(X_test.shape[0],1,X_test.shape[1])
    input_shape = (1,X_test.shape[2])
    num_classes = 1
    model = Sequential()
    model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True, input_shape=input_shape)))
    model.add(Dropout(0.25))
    model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
    model.add(Dropout(0.25))
    model.add(Bidirectional(CuDNNLSTM(64)))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes, activation='tanh'))
    model.compile(loss=mean_squared_error, optimizer = Adam(lr=0.01), metrics=['mape'])
    history = model.fit(X_train, Y_train.values, epochs = epochs, steps_per_epoch=int(len(X_train) / batch_size), validation_data=(X_test, Y_test.values), validation_steps=max(int(len(X_test)/batch_size),1), callbacks = [MetricsCheckpoint('logs')])
    #history = model.fit(X_train, Y_train.values, epochs = epochs, batch_size = batch_size, validation_data=(X_test, Y_test.values), callbacks = [MetricsCheckpoint('logs')])
    score = model.evaluate(X_test, Y_test.values)
    plotKerasLearningCurve()
    plt.show()  
    plot_learning_curve(history)
    plt.show()
    return model
    
#def f2(data):
    #maximize sales, by adjusting sys_hrs and mgr_hrs for f1, output (sys_hrs,mgr_hrs)


In [ ]:
split_date = '2018-01-01'
X_train, Y_train, X_test, Y_test = df[df.index < split_date]['STORE'], df[df.index < split_date]['SYSTEM_SCHED_HOURS'], df[df.index> split_date]['STORE'], df[df.index > split_date]['SYSTEM_SCHED_HOURS']
X_train = X_train.values.reshape(-1,1)
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.values.reshape(-1,1)
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
Y_concat = Y_train.values.reshape(-1,1)
Y_concat = Y_concat.reshape(Y_concat.shape[0], 1, Y_concat.shape[1])
X_concat = np.concatenate((X_train, Y_concat), axis = 2)
X_concat = np.concatenate((X_concat, Y_concat), axis = 2)
X_concat

In [ ]:
#Feature visaulization isn't done like this. Look below.

def generator(f1_model, store,epochs):
    #inputs are the date store. We generate o/p of system schedule hrs.(Since we need to determine the effect of changes in manager scheduled hours)
    batch_size = 16
    split_date = '2018-01-01'
    X_train, Y_train, X_test, Y_test = data[data.index < split_date]['STORE'], data[data.index < split_date]['SYSTEM_SCHED_HOURS'], data[data.index> split_date]['STORE'], data[data.index > split_date]['SYSTEM_SCHED_HOURS']
    X_train = X_train.values.reshape(-1,1)
    X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
    X_test = X_test.values.reshape(-1,1)
    X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
    input_shape = (1,X_test.shape[2])
    num_classes = 1
    model = Sequential()
    model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True, input_shape=input_shape)))
    model.add(Dropout(0.25))
    model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
    model.add(Dropout(0.25))
    model.add(Bidirectional(CuDNNLSTM(64)))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes, activation='relu'))
    model_out_train = model.predict(X_train)
    model_out_test = model.predict(X_test)
    Y_concat_train = model_out_train.values.reshape(-1,1)
    Y_concat_train = Y_concat_train.reshape(Y_concat_train.shape[0], 1, Y_concat_train.shape[1])
    X_concat_train = np.concatenate((X_train, Y_concat_train), axis = 2)
    X_concat_train = np.concatenate((X_concat_train, Y_concat_train), axis = 2)
    Y_concat_test = model_out_test.values.reshape(-1,1)
    Y_concat_test = Y_concat_test.reshape(Y_concat_test.shape[0], 1, Y_concat_test.shape[1])
    X_concat_test = np.concatenate((X_test, Y_concat_test), axis = 2)
    X_concat_test = np.concatenate((X_concat_test, Y_concat_test), axis = 2)
    Y_ones_train, Y_ones_test = np.ones(len(X_train)), np.ones(len(X_test))
    model.compile(loss=mean_squared_error(f1_model.predict(X_concat_train),Y_ones_train), optimizer = Adam(lr=0.01), metrics=['mape']) #error needs to be error between f1_model and 

In [ ]:
lstm_model = f1(df, 203, 70)

In [ ]:
#General scheme for feature visaulization is to iterate over all possible inputs
store_no = 203
max_input = [[[store_no, 0.0,0.0]]]
for i in range(24*7*4):
    For j in range(24*7*4):
        if (lstm_model.predict([[[store_no, float(i)/4.0, float(j)/4.0]]])[0]>lstm_model.predict(max_input)[0]):
            max_input = [[[store_no, float(i)/4.0, float(j)/4.0]]]
            
